# Kari

Introduces the main ways of using Kari.

## Training

Import the neccecary modules

In [ ]:
from utils_parameter_parsing import *
from sequence_processing_model import SequenceProcessingModel

Load the parameters from the config file and create a model. Note that you can also overide any arguments in `config.ini` by passing them in a dictionary to `process_parameters`.

In [ ]:
config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {}) # process_parameters expects a dictionary

Create a `SequenceProcessingModel` object. This object coordinates training/prediction/loading of models and datasets.

In [ ]:
sequence_processer = SequenceProcessingModel(**parameters)

We can then load the dataset (specified by the `dataset_text_folder` parameter). If pretrained token embeddings were specified with `token_pretrained_embedding_filepath`, these will be loaded also.

In [ ]:
sequence_processer.load_dataset()

Lastly, we specify the model we would like to use. Models are specified by the `model_name` parameter.

In [ ]:
sequence_processer.specify_model()

In [ ]:
sequence_processer.fit()

In [ ]:
np.count_nonzero((pred_idx == gold_idx)) / len(pred_idx)
# gold_idx[:18]

In [ ]:
sequence_processing_model.ds.tag_type_to_index

In [ ]:
neg_tag = sequence_processing_model.ds.tag_type_to_index['O']

In [ ]:
gold_labels = gold_idx != neg_tag
gold_labels[:18]

In [ ]:
pred_labels = pred_idx != neg_tag
pred_labels[:18]

In [ ]:
TP = 0
FP = 0
FN = 0

for pred, gold in zip(pred_labels, gold_labels):
    # FN
    if gold and not pred:
        FN += 1
    # FP
    if not gold and pred:
        FP += 1
    # TP
    if gold and pred:
        TP += 1
        
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
        
    

In [ ]:
TP, FP, FN

In [ ]:
precision, recall, f1

## Misc

In [ ]:
from utils_parameter_parsing import *
from sequence_processing_model import SequenceProcessingModel

config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {'model_name': 'MT-LSTM-CRF'}) # process_parameters expects a dictionary

sequence_processer = SequenceProcessingModel(**parameters)

In [ ]:
sequence_processer.load_dataset()

In [ ]:
sequence_processer.specify_model()

In [ ]:
sequence_processer.fit()

## Visulize the model

We can easily create an image depicted our model. First, install the [graphviz graph library](http://www.graphviz.org/) and the [Python interface](https://pypi.python.org/pypi/graphviz).

> More info can be found [here](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/).

In [ ]:
# set this variable equal to your Keras model object.
model_ = sequence_processer.model.model[0]

We can either: create and save an image on our local machine,

In [ ]:
from keras.utils import plot_model
plot_model(model_, to_file='model.png')

or, visulize it directly in the notebook

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_).create(prog='dot', format='svg'))